In [5]:
import sys
import os

sys.path.append(sys.path[0]+"/tracker")
sys.path.append(sys.path[0]+"/tracker/model")

from track_anything import TrackingAnything
import random 
import string
from dataset.longdataset import LongVideoDataset
from tools.runner import run_model_on_longVOS_set

In [2]:
# Change path to .datasets own folder 
if os.name == 'posix':
    longvos_root = '../data.nosync/LongVOS/test'
else:
    longvos_root = 'D:\HADA\data/LongVOS/test'


In [6]:
SAM_checkpoint = "./checkpoints/sam_vit_h_4b8939.pth"
if 'HQ' in sys.prefix.split('.')[-1]:SAM_checkpoint = "./checkpoints/sam_hq_vit_h.pth"
xmem_checkpoint = "./checkpoints/XMem-s012.pth"

In [7]:
runtimeargs_lst = [ 
{
    'DatasetArgs' :{
        'Dataset' : 'LongVOS',
        'Set' : 'test',
    },
      'TrackingAnythingArgs' : {
            'use_refinement' : False,
            'refinement_mode' : '_',
            'addArgs1':'_'
    }
}]
VideoLoader = LongVideoDataset(root_folder=longvos_root, filelist= f"{runtimeargs_lst[0]['DatasetArgs']['Set']}.txt")
for runtimeargs in runtimeargs_lst:
    runname = '{}{}{}{}_{}_{}'.format(
        'Refined' if runtimeargs['TrackingAnythingArgs']['use_refinement']  else 'XMEM',
        '_HQ' if 'HQ' in sys.prefix.split('.')[-1] else '_SAM',
        '_'+ runtimeargs['TrackingAnythingArgs']['refinement_mode'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] else '',
        '_' + runtimeargs['TrackingAnythingArgs']['addArgs1'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] and runtimeargs['TrackingAnythingArgs']['addArgs1'] != '' else '',
        'Davis_{}_{}_{}'.format(runtimeargs['DatasetArgs']['Year'],runtimeargs['DatasetArgs']['Set'],runtimeargs['DatasetArgs']['Resolution']) if runtimeargs['DatasetArgs']['Dataset'] == 'Davis' else runtimeargs['DatasetArgs']['Dataset'],
        ''.join(random.choice(string.ascii_letters) for _ in range(5))
    )
    print(f'Running Test: {runname}')
    model = TrackingAnything(SAM_checkpoint, xmem_checkpoint, None,runtimeargs['TrackingAnythingArgs'], save_inner_masks_folder = runname)
    masks, logits, painted_images = run_model_on_longVOS_set(name = runname,model = model,videoLoader = VideoLoader,compute_metrics = False, save_masks=True, compute_video=True,verbose = False)

Seq names: ['TVfoBiU2', 'R3UJGYwy', '81Oju1e7', 'JefVS4hq', 'FmzvB44A', 'shW04XAm', 'PhiOlFlA', 'w3XiaDdm', 'gUca15ef', 'gw84JOqH', 'bDv75KJl', 'irsQQbUO', 'B9fFIrgq', 'YMRntd88', 'T4pEUdxQ', 'UtEnRpiP', 'RCgebO9V', 'ealukzgh', 'LWVlKBlK', 'UBITNvZb', 'qeD7QcDW', 'Oj1z5fO0', 'Y6tOZtQV', 'UykV25jo', '6mNj04tC', 'GmfSNNa3', 'AIv9mv7T', 'aSNxf5ms', '58jpr19X', 'Kz1wgslb', '058oeZ2p', 'vlKROEmy', '5NImTLT6', 'T9woxlyM', '8kv99Cop', 'yxCvN6OJ', 'xL5OHe3Y', 'yfzVCnvU', 'HQ2UJdrW', 'Gz3GEPPC', 'tAyQIobn', 'yWnanBID', '7im4LEcb', '711gAS21', 'YHUoFe2u', '5VwnMLaz', 'MI0uwoMt', 'R4xwHrdP', 'GchnEETZ', 'PQPOuFNO']
Running Test: XMEM_SAM_LongVOS_uLiys
Initializing BaseSegmenter to cuda:0
Hyperparameters read from the model weights: C^k=64, C^v=512, C^h=64
Single object mode: False
Point Type convertion not specified


Tracking image:  11%|█         | 56/498 [00:04<00:29, 14.83it/s]

: 